In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.database import SessionLocal, engine, Base
from app.models import Fencer, Tournament, Bout
import datetime
from selenium.common.exceptions import NoSuchElementException

In [3]:
# Base.metadata.drop_all(engine)

In [4]:
# Base.metadata.create_all(engine)

In [3]:
from web_scrapping_helpers import FIEPage, parse_metadata, parse_pool, pool_to_db, tournament_to_db, parse_name, round_to_db

In [65]:
Scraper = FIEPage('epee', 's', 'A', 2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\liamc\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [51]:
Scraper.open_competions().previous().age_group('s').comp_category('A').select_weapon('epee').select_gender('m').select_event_type().search()


In [52]:
Scraper.to_results().to_pools().get_pools_data()

[]

In [53]:
Scraper.get_pool_names()

[]

In [8]:
metadata = Scraper.get_competitions()

In [4]:
def parse_tournament(metadata, Scraper, session):
    comp_meta, link = metadata
    tournament = tournament_to_db(comp_meta, session)
    pool_data = Scraper.to_results().to_pools().get_pools_data()
    if len(pool_data) == 0:
        return 1
    pool_names = Scraper.get_pool_names()
    offset = 0
    for pool in pool_data:
        bouts, pool_size = parse_pool(pool, pool_names[offset:])
        offset += pool_size
        pool_to_db(bouts, tournament, session)
        
    session.commit()
    Scraper.to_prelims()
    prelim_data = set(Scraper.get_prelim_data())
    try:
        Scraper.next_round_prelim()
    except NoSuchElementException:
        Scraper.next_round_prelim(option=True)
        
    prelim_data2 = set(Scraper.get_prelim_data())
    prelim_bouts = prelim_data.union(prelim_data2)

    if prelim_data != prelim_bouts:
        rounds = [256, 128, 64]
    else:
        rounds = [128, 64]
    prelim_bouts = list(prelim_bouts)

    pos = 0
    for rnd in rounds:
        num_bouts = rnd//2
        round_to_db(prelim_bouts[pos:pos+num_bouts], f"P{rnd}", tournament, session)
    """Prelim Parsing Logic"""
    Scraper.to_tableu()
    tableu_data = Scraper.get_de_data()
    for i in range(2):
        Scraper.next_round_de().next_round_de()
        next_round_data = Scraper.get_de_data()
        tableu_data.extend(next_round_data)
    rounds = [64, 32, 16, 8, 4, 2]
    pos = 0
    for rnd in rounds:
        num_bouts = rnd//2
        round_to_db(tableu_data[pos:pos+num_bouts], str(rnd), tournament, session)
        #print(f"Round {rnd}:\n{tableu_data[pos:pos+num_bouts]}")
        pos += num_bouts


    session.commit() 
    return 0



In [7]:
Scraper.

SyntaxError: invalid syntax (2098345420.py, line 1)

In [5]:
def scrape_cat(cat):
    Scraper = FIEPage()
    Scraper.open_competions().previous().age_group('s').comp_category(cat).select_weapon('epee').select_gender('m').select_event_type().search()
    session = SessionLocal
    session.begin()
    tries = 0
    while tries < 3:
        metadata = Scraper.get_competitions()

    
        for md in metadata:
            if tries < 3:
                m, link = md
                Scraper.open_window(link)
                main, new = Scraper.driver.window_handles
                Scraper.driver.switch_to.window(new)
                handles = Scraper.driver.window_handles
                tries += parse_tournament(md, Scraper, session)
                Scraper.driver.close()
                Scraper.driver.switch_to.window(main)
        Scraper.next_page_tournaments()
        
    

In [6]:
scrape_cat("GP")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\liamc\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


['YOO', 'Justin']
['MATSUMOTO', 'Ryu']
['CIMINI', 'Gabriele']
['SELIM', 'Ahmed']
['SCHMIDT', 'Richard']
['AHMED', 'Saif']
['SVYSTIL', 'Vitaliy']
['GABER', 'Mohamed']
['JEONG', 'Seyun']
['BRINKMANN', 'Marco']
['ROA', 'Hernando']
['PAOLINI', 'Giacomo']
['KOCH', 'Mate', 'Tamas']
['KOOVAKKAL', 'JOSHY', 'Aloshious']
['PHILIPPE', 'Arthur']
['HOUSIEAUX', 'Marc']
['ASHRAF', 'Mohamed']
['FUHRIMANN', 'Jonathan']
['VALCARCEL', 'Victor']
['VUORINEN', 'Niko']
['BUZZACCHINO', 'Gianpaolo']
['SYCH', 'Yan']
['CUPR', 'Michal']
['AL', 'ZAROONI', 'Khalifa']
['ODEH', 'Iyad']
['LEE', 'Seunghyun']
['NASSIF', 'Moussa']
['ZAWROTNIAK', 'Radoslaw']
['NIGGELER', 'Michele']
['KIM', 'Myeongki']
['PIATTI', 'Enrico']
['SHABAN', 'Mohanad']
['GALLY', 'Aymerick']
['MATEVOSOV', 'Artem']
['HO', 'Hon', 'Kiu']
['GAROZZO', 'Enrico']
['KIM', 'Jaewon']
['TROFYMENKO', 'Pavel']
['HAMED', 'Ahmed']
['ALMHEIRI', 'Masoud']
['HADZIC', 'Alen']
['HERZBERG', 'Fabian']
['CHOTIN', 'Titouan']
['BRUNOLD', 'Bruce']
['TALIOTIS', 'Alexandros']

In [3]:
session = SessionLocal


In [19]:

res = session.query(Fencer).filter_by(first_name='ADAM', last_name = 'MACZIK').where()

In [27]:
res[0].bout_lose[16].tournament_id

30

In [5]:
tourn = session.query(Tournament).filter_by(id = 30).first()

In [22]:
res = session.execute("SELECT * FROM fencer WHERE first_name = 'ADAM' & last_name = 'MACZIK'")

In [23]:
res.all()

[]

In [18]:
res[0].bout_lose

[ZHENGNAN CAI 5 to IAN SANDERS 2,
 YU TING WU 1 to IAN SANDERS 0,
 BALINT BAKOS 15 to IAN SANDERS 9]

In [10]:
Scraper = FIEPage()
Scraper.open_competions().previous().age_group('s').comp_category('A').select_weapon('epee').select_gender('m').select_event_type().search()
metadata = Scraper.get_competitions()
m, link = metadata[0]
Scraper.open_window(link)
handles = Scraper.driver.window_handles

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\liamc\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [19]:
Scraper.driver.window_handles
Scraper.driver.switch_to.window(handles[0])

In [21]:
Scraper.open_window(metadata[1])

ValueError: too many values to unpack (expected 2)

In [18]:
Scraper.driver.close()

In [10]:
session = SessionLocal
session.begin()

In [11]:

parse_tournament(metadata[8], Scraper, session)

['BARDENET', 'Alexandre']
['JENSEN', 'August', 'Lindstroem']
['WEBER', 'Frederik']
['HABRAN', 'Maxime']
['SVENSSON', 'Jonathan']
['DIABATE', 'Keletigui', 'Julien']
['CORDEIRO', 'Joao']
['TSINIS', 'Alexander']
['ANOKHIN', 'Vadim']
['CHAU', 'Chak', 'Hin', 'Nicholas']
['BRINCK-CROTEAU', 'Maxime']
['REDLI', 'Andras']
['RIVERA', 'DAVID', 'Paul']
['CIOBANU', 'Radu']
['REIZLIN', 'Igor']
['LE', 'QUANG', 'HUY', 'Phileas']
['BISHOP', 'William']
['KOMATA', 'Akira']
['HENRIQUEZ', 'ORTIZ', 'Reynier']
['ALHUSSAIN', 'Ahmad', 'Abbas', 'I.']
['EL', 'HAOUARI', 'Abdelkarim']
['GUSEV', 'Dmitriy']
['ANUPONGKUNKIT', 'Kantaphat']
['SCHANZLEH', 'Thom']
['FABREGAT', 'Angel']
['UYAMA', 'Satoru']
['SHARLAIMOV', 'Vadim']
['ERBETTA', 'Diego']
['NYCZ', 'Mateusz']
['KEPPEL', 'Colin']
['BOROSAK', 'Hrvoje']
['JUENG-AMNUAYCHAI', 'Korakote']
['FREILICH', 'Yuval', 'Shalom']
['ROBL', 'Troels', 'Christian']
['BAKOS', 'Balint']
['FICHERA', 'Marco']
['SZENYI', 'Peter']
['BERNARD', 'Florian']
['ADAIKIN', 'Dmitriy']
['CAMARGO'

In [23]:
fencer = session.query(Fencer).filter_by(last_name='GROVER').first()

In [18]:
session.query(Fencer).filter_by(first_name = 'JESUS ANDRES').all()[0].bout_win

[JESUS ANDRES LUGONES RUGGERI 5 to KASPER TAFENAU 3,
 JESUS ANDRES LUGONES RUGGERI 3 to LUCA HUSEMANN 1,
 JESUS ANDRES LUGONES RUGGERI 5 to GERARD GONELL 4,
 JESUS ANDRES LUGONES RUGGERI 5 to ADRIEN THEIN-SANDLER 0,
 JESUS ANDRES LUGONES RUGGERI 5 to RADOSLAW ZAWROTNIAK 4,
 JESUS ANDRES LUGONES RUGGERI 15 to MARCO BRINKMANN 14,
 JESUS ANDRES LUGONES RUGGERI 15 to SEDAT BAKAY 9,
 JESUS ANDRES LUGONES RUGGERI 15 to MARCO BRINKMANN 14,
 JESUS ANDRES LUGONES RUGGERI 15 to MARCO BRINKMANN 14,
 JESUS ANDRES LUGONES RUGGERI 15 to FREDERIK VON DER OSTEN 6]

In [17]:
session.query(Fencer).filter_by(country = 'CAN').all()[0].last_nameh

'GALLAGHER PELLETI...'

In [9]:
with SessionLocal as session, session.begin():
    tourn_meta, link = comp
    tournament = tournament_to_db(tourn_meta, session)
    main = Scraper.open_window(link)
    pool_data = Scraper.to_results().to_pools().get_pools_data()
    for pool in pool_data:
        bouts = parse_pool(pool)
        pool_to_db(bouts, tournament, session)





In [ ]:
for fencer in test_cases:
    first_names = []
    last_names = []
    for name in fencer:
        if name.isupper():
            name = ''.join([name[0], name[1:].lower()])
            last_names.append(name)
        else: first_names.append(name)
    first_name = ' '.join(first_names)
    last_name = ' '.join(last_names)
    print(first_name, last_name)


In [20]:
# session = SessionLocal
# session.begin()
fencer_test = session.query(Fencer).filter_by(first_name = 'SAMUEL').all()

In [22]:
fencer_test[0]

SAMUEL UNTERHAUSER, GER